# Single Model Training

This notebook is to enable model training for a single model using the specified parameters in config.yaml.


In [1]:
# Import required libraries
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "..",))
sys.path.append(project_root)
from src.pipelines.xgboost.train_single_model import main

In [2]:
# Call main pipeline
main()

INFO:src.pipelines.xgboost.train_single_model:
        XGBoost Model Training Pipeline is starting...
        
INFO:src.core.clients.bigquery:BigQuery client initialized for project: porygon-pipelines
/Users/anapreciado/Desktop/porygon-demand-forecasting/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
INFO:src.core.clients.bigquery:Loaded 148964 rows from table: walmart_master_table
INFO:src.pipelines.xgboost.train_single_model:Filtering outliers ...
INFO:src.pipelines.xgboost.train_single_model:Training model...
INFO:src.core.models.xgboost:
            Training XGBoost Model...
            


[0]	validation_0-poisson-nloglik:58.72804	validation_1-poisson-nloglik:44.50367
[1]	validation_0-poisson-nloglik:57.43633	validation_1-poisson-nloglik:42.91610
[2]	validation_0-poisson-nloglik:56.46433	validation_1-poisson-nloglik:41.74162
[3]	validation_0-poisson-nloglik:55.72748	validation_1-poisson-nloglik:40.86987
[4]	validation_0-poisson-nloglik:55.16653	validation_1-poisson-nloglik:40.22326
[5]	validation_0-poisson-nloglik:54.73864	validation_1-poisson-nloglik:39.74559
[6]	validation_0-poisson-nloglik:54.41210	validation_1-poisson-nloglik:39.39521
[7]	validation_0-poisson-nloglik:54.16307	validation_1-poisson-nloglik:39.14086
[8]	validation_0-poisson-nloglik:53.97346	validation_1-poisson-nloglik:38.95880
[9]	validation_0-poisson-nloglik:53.82940	validation_1-poisson-nloglik:38.83094
[10]	validation_0-poisson-nloglik:53.72022	validation_1-poisson-nloglik:38.74344
[11]	validation_0-poisson-nloglik:53.63771	validation_1-poisson-nloglik:38.68570
[12]	validation_0-poisson-nloglik:53.5

INFO:src.core.models.xgboost:Model saved to /Users/anapreciado/Desktop/porygon-demand-forecasting/src/artifacts/models/xgboost/20260122_114312/model_0.joblib
INFO:src.core.models.xgboost:
            Model Training Pipeline completed successfully!
            Run ID: /Users/anapreciado/Desktop/porygon-demand-forecasting/src/artifacts/models/xgboost/20260122_114312
            Artifacts saved to: /Users/anapreciado/Desktop/porygon-demand-forecasting/src/artifacts/models/xgboost/20260122_114312
            
